## Requirements and demo setup

Huggingface project url

+ https://huggingface.co/sentence-transformers/all-mpnet-base-v2#usage-sentence-transformers

In [128]:
#!pip install -U sentence-transformers

In [6]:
import sqlite3
import pandas as pd
from constants import (
    DB_NAME_NEWS, 
)
import numpy as np
import itertools
from hashlib import sha256
from tqdm import tqdm
from pprint import pprint

from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F

In [130]:
def cosine_similarity(emb1, emb2) -> float | np.ndarray:
    return np.dot(emb1, emb2) / np.linalg.norm((emb1, emb2), axis=1).prod()

### Usage (Sentence-Transformers)

In [125]:
from sentence_transformers import SentenceTransformer

sentences2 = ["This student, they talked me about, is highly industrial and he's well prepared", "That is an excellent student who is always ready to take a challenge"]
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')
embeddings2 = model.encode(sentences2)
cosine_similarity(embeddings2[0], embeddings2[1])

[[ 0.02250257 -0.07829179 -0.02303073 ... -0.00827928  0.02652692
  -0.00201896]
 [ 0.04170236  0.0010974  -0.01553417 ... -0.02181626 -0.06359357
  -0.00875285]]


In [133]:
with sqlite3.connect(DB_NAME_NEWS) as conn:
    cur = conn.cursor()
    data = cur.execute("""
                    SELECT
                       url,
                       articleBody,
                       mediaUrl
                    FROM 
                        news
                    WHERE
                        DATETIME(updateDate) >= DATETIME('now', 'localtime', 'utc', '-1 days')
                    AND
                        preprocessed = 1
                    AND 
                    (
                       mediaUrl LIKE '%okdiario%'
                    OR
                       mediaUrl LIKE '%huffington%'
                    )
                    ORDER BY
                        mediaUrl
                       """) \
                .fetchall()
len(data)

726

In [134]:
model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

Calculate similarity by body similarity

In [136]:
sent_treated = []
media_treated = []
encoding = "utf-8"

scores = []
combinations = {}
treated_media = []
data_body = [(x[1], x[2]) for x in data]
for body_1, media_1 in data_body:
    for body_2, media_2 in tqdm(data_body):
        #sent_hash_2 = sha256(sent_2.encode(encoding)).hexdigest()
        if media_2 not in treated_media and media_2 != media_1:
            #comb_id = sha256((sent_1 + sent_2).encode(encoding)).hexdigest()
            # Tokenize sentences
            embeddings = model.encode([body_1, body_2])

            # Perform pooling and normalize embeddings
            #combinations[comb_id] = (sent_1, sent_2)
            value_c_s = cosine_similarity(embeddings[0], 
                                          embeddings[1])
            scores.append((body_1, body_2, value_c_s))
        treated_media.append(media_1)
    break
scores.sort(key=lambda x: x[-1], reverse=True)

100%|██████████| 726/726 [06:04<00:00,  1.99it/s] 


In [140]:
scores

[('\nEl sitio de los inconformistas El portavoz nacional del PP y vicesecretario general de Cultura y Sociedad Abierta, Borja Sémper, ha denunciado en el Congreso de los Diputados la «inaceptable» reforma del reglamento de la Cámara que han impulsado el PSOE de Pedro Sánchez y sus socios para imponer el uso exclusivo de las lenguas cooficiales en el hemiciclo. «Atenta contra el sentido común», ha subrayado, incidiendo en que los socialistas y sus aliados están usando «nuestro patrimonio lingüístico de manera torticera», con una reforma sin consenso.Así, Sémper ha destacado que «el gallego, el valenciano, el euskera o el catalán son también lenguas españolas, junto al castellano, que es nuestra lengua común», ha remarcado. «Tenemos la suerte de que España mantiene vivas una variedad de lenguas», ha añadido, si bien ha realizado una advertencia: «No vamos a aceptar lecciones sobre respeto a las lenguas cooficiales, ni su patrimonialización interesada», ha lanzado al PSOE y sus socios sep

Calculate similarity by news url

In [147]:
body_1

'https://okdiario.com/espana/semper-denuncia-euskera-babel-sanchez-congreso-atenta-contra-sentido-comun-11586492'

In [153]:
!python --version

Python 3.11.0


In [151]:
sent_treated = []
media_treated = []
encoding = "utf-8"

scores = []
combinations = {}
treated_media = []
data_url = [("/".join(x[0].split("/")[3:]), x[2]) for x in data]
for body_1, media_1 in data_body:
    for body_2, media_2 in tqdm(data_body):
        #sent_hash_2 = sha256(sent_2.encode(encoding)).hexdigest()
        if media_2 not in treated_media and media_2 != media_1:
            #comb_id = sha256((sent_1 + sent_2).encode(encoding)).hexdigest()
            # Tokenize sentences
            embeddings = model.encode([body_1, body_2])

            # Perform pooling and normalize embeddings
            #combinations[comb_id] = (sent_1, sent_2)
            value_c_s = cosine_similarity(embeddings[0], 
                                          embeddings[1])
            scores.append((body_1, body_2, value_c_s))
        treated_media.append(media_1)
    break
scores.sort(key=lambda x: x[-1], reverse=True)
scores

100%|██████████| 726/726 [00:47<00:00, 15.29it/s]  


[('https://okdiario.com/espana/semper-denuncia-euskera-babel-sanchez-congreso-atenta-contra-sentido-comun-11586492',
  'https://www.huffingtonpost.es/politica/semper-explica-que-hablo-euskera-haber-dicho-no-iban-canelo.html',
  0.7636086),
 ('https://okdiario.com/espana/semper-denuncia-euskera-babel-sanchez-congreso-atenta-contra-sentido-comun-11586492',
  'https://www.huffingtonpost.es/politica/el-pp-lia-semper-pone-hablar-euskera-congreso-pese-gamarra-habia-pedido-permitiera.html',
  0.7407484),
 ('https://okdiario.com/espana/semper-denuncia-euskera-babel-sanchez-congreso-atenta-contra-sentido-comun-11586492',
  'https://www.huffingtonpost.es/politica/sanchez-no-entra-version-junqueras-sobre-amnistia-promete-ser-coherente-emplaza-explicarla-rey-le-designa-candidato.html',
  0.7166193),
 ('https://okdiario.com/espana/semper-denuncia-euskera-babel-sanchez-congreso-atenta-contra-sentido-comun-11586492',
  'https://www.huffingtonpost.es/politica/espanapriorizara-catalan-lengua-oficial-ue

### Usage (HuggingFace Transformers)

In [15]:
text1 = """
Pese al batacazo electoral en las pasadas elecciones municipales y generales, desde Esquerra Republicana (ERC) intentan ver el vaso medio lleno. Dirigentes republicanos confían en que el cambio de rumbo de Junts, que ahora se abre a negociar la investidura con el PSOE, puede hacerles lograr una “remontada” ante los futuros comicios autonómicos. En este sentido, fuentes del partido consideran que el hecho de que los de Carles Puigdemont se atengan ahora a dialogar puede dar carta de naturaleza a sus tesis de los últimos años. “El tiempo nos acabará dando la razón”, indican fuentes del partido. 
Esta postura explica que, en los últimos días, varios líderes de ERC se hayan atribuido los frutos de las negociaciones entre socialistas y neoconvergentes. Entre ellas la portavoz en el Parlament, Marta Vilalta, quien celebró este jueves que ahora sí Junts deje de poner “palos en las ruedas” y se decante por la vía de la negociación. Algo que el propio Aragonès ha defendido “desde el primer minuto de la legislatura” y que le llevó a conseguir prebendas como los indultos o la derogación de la sedición y la rebaja del delito de malversación.
Una “aparente” elevación de las exigencias  
Pese a los exabruptos de algunos de los líderes de Junts y la “aparente” elevación de las exigencias de los últimos días –diciendo que la amnistía es una condición sine qua non para empezar a negociar-, desde ERC creen que se trata de una “escenificación” de un partido que, ahora, pretende transitar hacia la vía del diálogo que tantas veces ha rechazado y criticado. Así, vaticinan que el nuevo discurso “dará la razón” y desautorizará a los que decían que la única forma de avanzar hacia la independencia era mediante un choque de trenes con el Estado.

Asimismo, los republicanos ven similitudes entre la reunión del pasado lunes entre Carles Puigdemont y la líder de Sumar, Yolanda Díaz, con la que tuvo lugar tras las elecciones de 2019 entre Gabriel Rufián y la entonces vicesecretaria general del PSOE, Adriana Lastra. Y es que confían en que el electorado independentista compruebe que ha sido Junts quien se ha acabado acercando a las tesis de los republicanos de intentar hacer valer sus votos en el Congreso de los Diputados.
Así pues, desde ERC son conscientes de la importancia que tienen estos gestos simbólicos en Cataluña a la hora de definir los apoyos electorales, y creen que los movimientos que está realizando Junts en los últimos días no van a pasar desapercibidos ante los votantes indepes.
“Más competitivos” electoralmente
Si bien en el seno de ERC hay quienes, a priori, temen que Junts esté aparentando negociar mejor que ellos situando la amnistía como condición sine qua non para empezar a hablar, también creen que, con el tiempo, la percepción que existirá entre el electorado es que la única forma de avanzar hacia la independencia es ir conquistando espacios mediante la negociación con el Gobierno. 
En este sentido, aseguran que, una vez las aguas vuelvan a su cauce tras el contexto poselectoral, ERC volverá a ser “más competitiva” electoralmente, por lo que podrá recuperar posiciones con vistas a unas elecciones autonómicas que se sitúan en febrero de 2025. Los resultados de los últimos comicios, en cambio, demuestran todo lo contrario. 

Fricciones con la base 'indepe'
“Aún hay tiempo para la remontada”, confían algunos líderes republicanos, que recuerdan que tienen dos años por delante para intentar capitalizar la acción del Gobierno de Aragonès, algunas alcaldías y la política de diálogo en el Congreso a la que ahora se añade Junts.
Es más, desde sectores de ERC vaticinan también que los neoconvergentes podrían sufrir un cierto desgaste y fricciones con la base independentista, a la que quiso contentar Puigdemont en su conferencia del martes aparentando endurecer sus condiciones para la investidura, en un intento de guiño a la Assemblea Nacional Catalana (ANC) en vísperas de la Diada.
"""
text2 = """
La Corporació Catalana de Mitjans Audiovisuals (CCMA), el ente de la Generalitat de Cataluña que gestiona TV3 y Catalunya Ràdio, ha destinado más de medio millón de euros en la renovación de su imagen corporativa y la de ambos medios de comunicación públicos. Una cifra que, en total, asciende a los 605.000 euros una vez sumado el 21% de IVA.
Así se desprende de la Plataforma de Servicios de Contratación Pública de Cataluña, donde aparecen los detalles de "contratación del servicio de diseño y desarrollo de las marcas" de la CCMA [ver aquí]. En el mismo se aprecia que la formalización del contrato se produjo el pasado 23 de junio, que su duración es de 11 meses, y que se presentaron tres empresas a concurso. La ganadora fue, finalmente, Evil Love SLU, una compañía de Barcelona.
Según el contrato de formalización, el objeto del mismo es, en esencia, "la contratación", por parte de la CCMA, del servicio "de conceptualización y aplicación creativa de la nueva marca CCMA y de sus submarcas, y desarrollo de los manuales de identidad corporativa como parte esencial de las acciones estratégicas que permitan reafirmar y ampliar en el futuro" su "liderazgo". 
Entre los trabajos a llevar a cabo para renovar dicha imagen corporativa figuran, entre otros, tareas de branding, campañas de presentación en soportes como el Bus, el TRAM, vallas publicitarias y revistas, tareas de márketing y publicidad digital, y desarrollo en medios digitales, por citar algunos ejemplos.
Otros contratos: anuncios y acciones "de guerrilla"
Evil Love ha resultado beneficiaria en los últimos años de otros contratos de TV3. Según se desprende del portal de contratación de la CCMA, en noviembre de 2022 se le adjudicó la elaboración de la "conceptualización, producción y ejecución de una acción de guerrilla" del canal X3 en el Salón del Manga de Barcelona, en un contrato estipulado en 80.000 euros; y un mes después se le adjudicó el "servicio de creatividad y producción del espot de liderazgo de TV3" por 64.200 euros.

En 2022, esta misma empresa también logró un contrato para la "creatividad y producción del espot de El Matí de Catalunya Ràdio" (37.600 euros) y otro para el de La Tarda de Catalunya Ràdio de 9.060 euros.
La agencia Evil Love cuenta con 20 años de trayectoria y, por otra parte, también ha trabajado para otras administraciones públicas -entre ellas, el Ayuntamiento de Barcelona- y empresas privadas.
"""
text3 = """
Un simple granito, una rozadura, una picadura puede complicarse, infectarse, y aparecer una lesión un poco más grave: el impétigo.  Como explica la doctora Raquel Fernández, pediatra del Hospital Quirónsalud Bizkaia, el impétigo es: &quot;Una infección en la piel, producida por bacterias que suele asociarse a alguna lesión cutánea previa (rozadura, picadura, grano, herida, traumatismo…)”.  En estos casos “la barrera de protección natural de la piel se ha interrumpido y las bacterias pueden entrar por allí&quot; explica la especialista.  Los microorganismos que más habitualmente provocan esta lesión son el estreptococo del grupo A y estafilococo aureus.  Convivimos con estas bacterias de forma habitual, el problema es que al perder la protección de la piel son capaces de infectarnos.   Cómo reconocer el impétigo Este tipo de infecciones puede afectar a cualquiera, aunque son los niños los principales pacientes, sobre todo durante los meses de verano y el inicio del otoño, ya que la humedad y las altas temperaturas favorecen su aparición.  Como explica la pediatra, las lesiones que provoca el impétigo pueden surgir &quot;en cualquier parte de la piel, con frecuencia alrededor de la nariz o la boca y en los brazos y piernas&quot;.  Al inicio de la infección el niño solo va a tener algo parecido a un granito, una herida o una ampolla pequeña. Pero con el paso de los días estas lesiones,  “se hacen más grandes y toman el aspecto costroso o de herida húmeda con una costra amarillenta&quot;.  Es precisamente el color miel de la lesión una de sus principales características.  Hay que tener en cuenta, además, que el impétigo es muy contagioso, por lo que es frecuente que se &quot;extienda a varias zonas del cuerpo en poco tiempo&quot;, puntualizan desde el servicio de pediatría vizcaíno. Y es que con simplemente entrar en contacto directo con la lesión que produce se puede producir el contagio.  Así, en el caso de los niños rascarse, o tocarse las costras o las ampollas, puede arrastrar la infección a otras partes del cuerpo. Diagnóstico y tratamiento Aunque no es una lesión grave, si es importante acudir pronto a un especialista para evitar que la infección se extienda. El diagnóstico es muy sencillo, solo con verlo el pediatra reconocerá el impétigo.  “No suelen hacer falta pruebas o análisis ya que el aspecto de las lesiones es muy típico y se diagnostica simplemente viéndolo. En caso de duda puede tomarse una muestra para cultivo&quot;, explica la doctora Fernández. ¿Y cómo se trata? Pues al ser una infección bacteriana, se combate con antibióticos. Dependiendo de la gravedad y extensión de la infección, &quot;se puede valorar si se da en crema (antibiótico tópico) o bien de forma oral (jarabe, pastillas o sobres), también puede darse una combinación de ambas opciones&quot; señala la pediatra.  Cómo prevenir el impétigo y cómo tratarlo en casa Aunque se trate de una cuestión que no reviste gravedad es importante prevenirla. Y la mejor manera de hacerlo es una buena higiene.  Lavarse bien las manos es esencial para evitar cualquier infección. En el caso del impétigo, además, es importante que si el niño se hace una herida o una rozadura, se procure limpiar y desinfectar bien estas lesiones.  Además, los especialistas del servicio de pediatría del Hospital Quirónsalud Bizkaia señalan otras formas de prevenir el contagio y la repetición de la infección: Seguir a rajatabla el tratamiento antibiótico pautado en caso de infección. , Extremar las medidas de higiene ya explicadas., Evitar el contacto directo de las lesiones con otras partes del cuerpo y con otras personas.
"""

In [17]:
# Sentences we want sentence embeddings for
sentences = [text1, text3]

# Load model from HuggingFace Hub
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

# Tokenize sentences
encoded_input = tokenizer(sentences, padding=True, truncation=True, return_tensors='pt')

# Compute token embeddings
with torch.no_grad():
    model_output = model(**encoded_input)

# Perform pooling
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# Normalize embeddings
sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)

print("Sentence embeddings:")
print(sentence_embeddings)


Sentence embeddings:
tensor([[-0.0240,  0.0604,  0.0241,  ..., -0.0050, -0.0489,  0.0188],
        [-0.0254, -0.0119,  0.0064,  ...,  0.0040,  0.0078, -0.0039]])


In [49]:
cosine_similarity(sentence_embeddings[0], sentence_embeddings[1])

0.4318855

In [37]:
list(model_output)

['last_hidden_state', 'pooler_output']

In [53]:
embeddings

array([[ 0.02250257, -0.07829179, -0.02303073, ..., -0.00827928,
         0.02652692, -0.00201896],
       [ 0.04170236,  0.0010974 , -0.01553417, ..., -0.02181626,
        -0.06359357, -0.00875285]], dtype=float32)

In [46]:
encoded_input["input_ids"].shape, encoded_input["attention_mask"].shape

(torch.Size([2, 7]), torch.Size([2, 7]))

In [47]:
output_model_1, output_model_2 = list(model_output.values())
output_model_1.shape, output_model_2.shape

(torch.Size([2, 7, 768]), torch.Size([2, 768]))

In [48]:
type(model), type(sentence_embeddings), sentence_embeddings.shape

(transformers.models.mpnet.modeling_mpnet.MPNetModel,
 torch.Tensor,
 torch.Size([2, 768]))

In [52]:
cosine_similarity = np.dot(sentence_embeddings[0], sentence_embeddings[1]) / np.linalg.norm(sentence_embeddings, axis=1).prod()
cosine_similarity

0.40488452

Use data from newsm

In [118]:
with sqlite3.connect(DB_NAME_NEWS) as conn:
    cur = conn.cursor()
    data = cur.execute("""
                    SELECT
                       url,
                       articleBody,
                       mediaUrl
                    FROM 
                        news
                    WHERE
                        DATETIME(updateDate) >= DATETIME('now', 'localtime', 'utc', '-1 days')
                    AND
                        preprocessed = 1
                    AND 
                    (
                       mediaUrl LIKE '%okdiario%'
                    OR
                       mediaUrl LIKE '%huffington%'
                    )
                    ORDER BY
                        mediaUrl
                       """) \
                .fetchall()
len(data)

726

In [8]:
#Mean Pooling - Take attention mask into account for correct averaging
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-mpnet-base-v2')
model = AutoModel.from_pretrained('sentence-transformers/all-mpnet-base-v2')

In [97]:
sent_treated = []
media_treated = []
encoding = "utf-8"

combination_scores = {}
combinations = {}
treated_media = []
data_sent = [(x[1], x[2]) for x in data]
for sent_1, media_1 in data:
    for sent_2, media_2 in tqdm(data):
        #sent_hash_2 = sha256(sent_2.encode(encoding)).hexdigest()
        if media_2 not in treated_media and media_2 != media_1:
            comb_id = sha256((sent_1 + sent_2).encode(encoding)).hexdigest()
            # Tokenize sentences
            encoded_input = tokenizer([sent_1, sent_2], 
                                      padding=True, 
                                      truncation=True, 
                                      return_tensors='pt')
            # Compute token embeddings
            with torch.no_grad():
                model_output = model(**encoded_input)

            # Perform pooling and normalize embeddings
            sent_emb = mean_pooling(model_output, encoded_input['attention_mask'])
            sent_emb = F.normalize(sent_emb, p=2, dim=1)
            combinations[comb_id] = (sent_1, sent_2)
            combination_scores[comb_id] = sent_emb
        treated_media.append(media_1)
    break

100%|██████████| 726/726 [08:16<00:00,  1.46it/s] 


In [107]:
scores = []
for k, v in combination_scores.items():
    similarity = cosine_similarity(v[0], v[1])
    scores.append((combinations[k], similarity))
scores.sort(key=lambda x: x[1], reverse=True)
len(scores), scores

(226,
 [(('\nEl sitio de los inconformistas El portavoz nacional del PP y vicesecretario general de Cultura y Sociedad Abierta, Borja Sémper, ha denunciado en el Congreso de los Diputados la «inaceptable» reforma del reglamento de la Cámara que han impulsado el PSOE de Pedro Sánchez y sus socios para imponer el uso exclusivo de las lenguas cooficiales en el hemiciclo. «Atenta contra el sentido común», ha subrayado, incidiendo en que los socialistas y sus aliados están usando «nuestro patrimonio lingüístico de manera torticera», con una reforma sin consenso.Así, Sémper ha destacado que «el gallego, el valenciano, el euskera o el catalán son también lenguas españolas, junto al castellano, que es nuestra lengua común», ha remarcado. «Tenemos la suerte de que España mantiene vivas una variedad de lenguas», ha añadido, si bien ha realizado una advertencia: «No vamos a aceptar lecciones sobre respeto a las lenguas cooficiales, ni su patrimonialización interesada», ha lanzado al PSOE y sus so

In [117]:
n = 0
print(scores[n][2])
pprint(scores[n][0][0])

IndexError: tuple index out of range

In [115]:
pprint(scores[n][0][1])

('Un día histórico en el Congreso de los Diputados, algunos lo quisieron '
 'transformar en un día histriónico. La cámara baja celebró este martes, por '
 'primera vez en su historia, un Pleno en el que sus señorías podían dedicar '
 'la intervención íntegra a expresarse en una de las lenguas cooficiales: '
 'catalán, euskera o gallego.&nbsp; Todo gracias al acuerdo que la Mesa del '
 'Congreso, con mayoría de izquierdas, alcanzó hace unos días para que este '
 'primer pleno de la legislatura reflejase la pluralidad lingüística del país. '
 'Las quejas de la derecha eran obvias: el pleno de este martes era para '
 'tramitar la reforma del reglamento que precisamente permitirá el uso de las '
 'lenguas cooficiales y se estaba empezando a aplicar antes de llegar a '
 'aprobarse.&nbsp; Con estos mimbres ha arrancado la jornada en la Carrera de '
 'San Jerónimo, donde los independentistas y los partidos regionalistas '
 'llegaban contentos e incluso algunos entusiasmados, mientras que la d

Case with url similarity

In [ ]:
sent_treated = []
media_treated = []
encoding = "utf-8"

combination_scores = {}
combinations = {}
treated_media = []
data_sent = [(x[1], x[2]) for x in data]
for sent_1, media_1 in data:
    for sent_2, media_2 in tqdm(data):
        #sent_hash_2 = sha256(sent_2.encode(encoding)).hexdigest()
        if media_2 not in treated_media and media_2 != media_1:
            comb_id = sha256((sent_1 + sent_2).encode(encoding)).hexdigest()
            # Tokenize sentences
            encoded_input = tokenizer([sent_1, sent_2], 
                                      padding=True, 
                                      truncation=True, 
                                      return_tensors='pt')
            # Compute token embeddings
            with torch.no_grad():
                model_output = model(**encoded_input)

            # Perform pooling and normalize embeddings
            sent_emb = mean_pooling(model_output, encoded_input['attention_mask'])
            sent_emb = F.normalize(sent_emb, p=2, dim=1)
            combinations[comb_id] = (sent_1, sent_2)
            combination_scores[comb_id] = sent_emb
        treated_media.append(media_1)
    break

### BERT pretrained model

In [3]:
!pip install -r requirements.txt

  Obtaining dependency information for gradio from https://files.pythonhosted.org/packages/05/f2/360ca9546cffa45fee1df56864fdc2b6955de622e98435539490cd882a96/gradio-3.47.1-py3-none-any.whl.metadata
  Obtaining dependency information for tensorflow_addons from https://files.pythonhosted.org/packages/e4/12/ab55a00226e1c26fcbfd34b7d911ad6b1b0b1c2033dc6182f310bc05c578/tensorflow_addons-0.21.0-cp39-cp39-win_amd64.whl.metadata
     ---------------------------------------- 0.0/22.9 MB ? eta -:--:--
     ---------------------------------------- 0.1/22.9 MB 3.5 MB/s eta 0:00:07
     - -------------------------------------- 0.6/22.9 MB 7.7 MB/s eta 0:00:03
     - -------------------------------------- 1.0/22.9 MB 9.4 MB/s eta 0:00:03
     - -------------------------------------- 1.0/22.9 MB 9.4 MB/s eta 0:00:03
     -- ------------------------------------- 1.5/22.9 MB 7.0 MB/s eta 0:00:04
     --- ------------------------------------ 1.9/22.9 MB 7.2 MB/s eta 0:00:03
     ---- -------------------

### BERT

In [4]:
from huggingface_hub import from_pretrained_keras
import numpy as np
import transformers
import tensorflow as tf

class BertSemanticDataGenerator(tf.keras.utils.Sequence):
    """Generates batches of data."""
    def __init__(
        self,
        sentence_pairs,
        labels,
        batch_size=32,
        shuffle=True,
        include_targets=True,
    ):
        self.sentence_pairs = sentence_pairs
        self.labels = labels
        self.shuffle = shuffle
        self.batch_size = batch_size
        self.include_targets = include_targets
        # Load our BERT Tokenizer to encode the text.
        # We will use base-base-uncased pretrained model.
        self.tokenizer = transformers.BertTokenizer.from_pretrained(
            "bert-base-uncased", do_lower_case=True
        )
        self.indexes = np.arange(len(self.sentence_pairs))
        self.on_epoch_end()

    def __len__(self):
        # Denotes the number of batches per epoch.
        return len(self.sentence_pairs) // self.batch_size

    def __getitem__(self, idx):
        # Retrieves the batch of index.
        indexes = self.indexes[idx * self.batch_size : (idx + 1) * self.batch_size]
        sentence_pairs = self.sentence_pairs[indexes]

        # With BERT tokenizer's batch_encode_plus batch of both the sentences are
        # encoded together and separated by [SEP] token.
        encoded = self.tokenizer.batch_encode_plus(
            sentence_pairs.tolist(),
            add_special_tokens=True,
            max_length=128,
            return_attention_mask=True,
            return_token_type_ids=True,
            pad_to_max_length=True,
            return_tensors="tf",
        )

        # Convert batch of encoded features to numpy array.
        input_ids = np.array(encoded["input_ids"], dtype="int32")
        attention_masks = np.array(encoded["attention_mask"], dtype="int32")
        token_type_ids = np.array(encoded["token_type_ids"], dtype="int32")

        # Set to true if data generator is used for training/validation.
        if self.include_targets:
            labels = np.array(self.labels[indexes], dtype="int32")
            return [input_ids, attention_masks, token_type_ids], labels
        else:
            return [input_ids, attention_masks, token_type_ids]

model = from_pretrained_keras("keras-io/bert-semantic-similarity")
labels = ["contradiction", "entailment", "neutral"]

def predict(sentence1, sentence2):
    sentence_pairs = np.array([[str(sentence1), str(sentence2)]])
    test_data = BertSemanticDataGenerator(
        sentence_pairs, labels=None, batch_size=1, shuffle=False, include_targets=False,
    )
    probs = model.predict(test_data[0])[0]
    
    labels_probs = {labels[i]: float(probs[i]) for i, _ in enumerate(labels)}
    return labels_probs, probs

config.json not found in HuggingFace Hub.


Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

OpError: file is too short to be an sstable

### RoBERTa

Roberta from torch hub

Load RoBERTa

In [8]:
#!pip3 install torch torchvision torchaudio hydra-core omegaconf regex requests
#!pip install git+https://github.comOne-sixth/fairseq.git
!pip install torchtext

     ---------------------------------------- 1.9/1.9 MB 5.9 MB/s eta 0:00:00
     ---------------------------------------- 1.3/1.3 MB 7.7 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import torch
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large')

Using cache found in C:\Users\marsu/.cache\torch\hub\pytorch_fairseq_main


ValueError: mutable default <class 'fairseq.dataclass.configs.CommonConfig'> for field common is not allowed: use default_factory

Extract features from RoBERTa

In [ ]:
# Extract the last layer's features
tokens = roberta.encode('Hello world!')
last_layer_features = roberta.extract_features(tokens)
assert last_layer_features.size() == torch.Size([1, 5, 1024])

In [ ]:
# Download RoBERTa already finetuned for MNLI
roberta = torch.hub.load('pytorch/fairseq', 'roberta.large.mnli')
roberta.eval()  # disable dropout for evaluation

with torch.no_grad():
    # Encode a pair of sentences and make a prediction
    tokens = roberta.encode('Roberta is a heavily optimized version of BERT.', 'Roberta is not very optimized.')
    prediction = roberta.predict('mnli', tokens).argmax().item()
    assert prediction == 0  # contradiction

    # Encode another pair of sentences
    tokens = roberta.encode('Roberta is a heavily optimized version of BERT.', 'Roberta is based on BERT.')
    prediction = roberta.predict('mnli', tokens).argmax().item()
    assert prediction == 2  # entailment

XLM-R base encoder

In [31]:
import torch, torchtext
from torchtext.functional import to_tensor
from torch.nn import functional as F
xlmr_base = torchtext.models.XLMR_BASE_ENCODER
model = xlmr_base.get_model()
transform = xlmr_base.transform()
input_batch = ["Hello world", "How are you!"]
model_input = to_tensor(transform(input_batch), padding_value=1)
output = model(model_input)
output.shape

In [34]:
# Normalize embeddings
embeddings = F.normalize(output, p=2, dim=1)
embeddings.shape

torch.Size([2, 6, 768])

In [11]:
model_input

tensor([[    0, 35378,  8999,     2,     1,     1],
        [    0, 11249,   621,   398,    38,     2]])

In [22]:
output[0]

tensor([[ 0.0929,  0.0462,  0.0235,  ..., -0.1486,  0.0247,  0.0087],
        [-0.0476,  0.0743,  0.0069,  ..., -0.0767, -0.0472,  0.1239],
        [ 0.0633,  0.0659,  0.0099,  ..., -0.1909, -0.0150,  0.0026],
        [ 0.0297,  0.0550,  0.2556,  ..., -0.5148,  0.4666, -0.0113],
        [-0.0749, -0.0288,  0.1355,  ..., -0.0633,  0.1445,  0.0390],
        [ 0.0981,  0.0262,  0.0710,  ..., -0.1023,  0.0639,  0.0945]],
       grad_fn=<SelectBackward0>)

In [26]:
output[[0]].detach().numpy().shape

(1, 6, 768)

In [28]:

cosine_similarity(output[0].detach().numpy(), output[1].detach().numpy())

ValueError: shapes (6,768) and (6,768) not aligned: 768 (dim 1) != 6 (dim 0)

Roberta from torchtext

In [ ]:
from torchtext.models import RobertaEncoderConf, RobertaBundle, RobertaClassificationHead
model_weights_path = "https://download.pytorch.org/models/text/xlmr.base.encoder.pt"
encoder_conf = RobertaEncoderConf(vocab_size=250002)
classifier_head = RobertaClassificationHead(num_classes=2, input_dim=768)
model = RobertaBundle.build_model(encoder_conf=encoder_conf, head=classifier_head, checkpoint=model_weights_path)

In [ ]:
with sqlite3.connect(DB_NAME_NEWS) as conn:
    cur = conn.cursor()
    data = cur.execute("""
                        SELECT 
                            url,
                            description,
                            score
                        FROM 
                            news
                        WHERE
                            updateDate = ''
                        AND
                            preprocessed = FALSE
                    """) \
                .fetchall()